In [1]:
import getpass
import os

# Prompt for API key
#LLAMA_CLOUD_API_KEY = getpass("Enter your LLAMA CLOUD API key: ")
#ANTHROPIC_API_KEY = getpass("Enter your Antropic API key: ")
os.environ["GOOGLE_API_KEY"] = getpass.getpass()


 ········


In [2]:
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Retrieve OpenAI API key from environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    logging.error("Google Gemini API key not found. Please set the 'GOOGLE_API_KEY' environment variable.")
else:
    GOOGLE_API_KEY = GOOGLE_API_KEY

In [3]:

import google.generativeai as genai

# Configure the model using the API key from the previous cell
genai.configure(api_key=GOOGLE_API_KEY)

# Optional: Set generation configuration (modify as needed)
generation_config = {
    "temperature": 0.2,  # Controls randomness (1=more random, 0=less random)
    "top_p": 0.95,    # Probability distribution weighting
    "max_output_tokens": 8192, 
}

# Optionally create a GenerativeModel instance with specific settings
model = genai.GenerativeModel(
    # model_name="gemini-2.0-flash-exp",
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction=" ",
)

# Print confirmation message
print("Gemini model configured successfully!")

Gemini model configured successfully!


In [4]:
import os
import logging
import google.generativeai as genai

def merge_database_package_procedures_with_history(
    differences_file_path: str,
    heritage_package_procedure: str,
    geminia_package_procedure: str,
    output_package_path: str,
) -> str:
    """
    Merges procedures using Gemini with history and completion checks using 'FINAL_MERGING_DONE'.

    Args:
        differences_file_path: Path to the file containing differences between the procedures.
        heritage_package_procedure: Path to the heritage procedure file.
        geminia_package_procedure: Path to the geminia procedure file.
        output_package_path: Path to save the merged procedure.

    Returns:
        Path of the saved merged procedure.
    """

    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

    generation_config = {
        "temperature": 1,
        "top_p": 0.95,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash-exp",
        generation_config=generation_config,
        system_instruction="You are a senior PL/SQL developer proficient in merging procedures and functions. For the large procedures exceeding 500 lines, you will merge them by generating PL/SQL code in chunks iteratively until the last chunk. Indicate completion with 'FINAL_MERGING_DONE' at the bottom of the last chunk.",
    )

    try:
        with open(differences_file_path, "r") as diff_file:
            diff_content = diff_file.read()

        with open(heritage_package_procedure, "r") as heritage_file:
            heritage_content = heritage_file.read()

        with open(geminia_package_procedure, "r") as geminia_file:
            geminia_content = geminia_file.read()

        base_prompt = f"""
You are an expert PL/SQL developer. Your task is to merge two plsql procedures into a single harmonized procedure.
The differences between the two procedures are stored in here (you will only use this for context, dont paste it directly):
{diff_content}
The first procedure is here: Heritage Procedure:
{heritage_content}
The second procedure that will be used for comparison and merging is here: Geminia Procedure:
{geminia_content}
When the final output is done, write a new line below the final code saying "FINAL_MERGING_DONE" at the end of your response.
        """

        merged_chunks = []
        history = [{"role": "user", "parts": [base_prompt]}]
        chat_session = model.start_chat(history=history)

        while True:
            response = chat_session.send_message("continue")  # Always use "continue"
            chunk = response.text.strip()
            merged_chunks.append(chunk)
            history.append({"role": "model", "parts": [chunk]})

            if "FINAL_MERGING_DONE" in chunk.upper():
                merged_chunks[-1] = merged_chunks[-1].replace("FINAL_MERGING_DONE", "").strip()
                break
            else:
                history.append({"role": "user", "parts": ["Continue."]})
                chat_session = model.start_chat(history=history)

        merged_content = "\n\n".join(merged_chunks)
        output_dir = os.path.dirname(output_package_path) or "."
        os.makedirs(output_dir, exist_ok=True)

        with open(output_package_path, "w") as output_file:
            output_file.write(merged_content)

        logging.info(f"Merged package written to {output_package_path}")
        return output_package_path

    except Exception as e:
        logging.error(f"LLM Error: {e}")
        raise RuntimeError(f"Failed to merge using LLM: {e}")

# Example usage
differences_file_path = "diffs/GIN_STP_PKG/procedures_gin_policies_prc_diff.txt"
heritage_package_procedure = "packages/GIN_STP_PKG_HERITAGE_BODY/procedures/gin_policies_prc.sql"
geminia_package_procedure = "packages/GIN_STP_PKG_NEW_GEMINIA_BODY/procedures/gin_policies_prc.sql"
output_package_path = "tests/merged_procedure.sql"

# Call the function to merge procedures
merged_package_path = merge_database_package_procedures_with_history(
    differences_file_path,
    heritage_package_procedure,
    geminia_package_procedure,
    output_package_path
)
print(f"Merged procedure saved to: {merged_package_path}")


2025-01-06 10:19:47,707 - INFO - Merged package written to tests/merged_procedure.sql


Merged procedure saved to: tests/merged_procedure.sql
